#### _OTECH-2685-create-and-update-public.encr_

https://hintmd.atlassian.net/browse/OTECH-2685

#### _part 1 : write python utility to load data governance csv into s3_

In [1]:
_pocnumber = '0.0.5'

In [2]:
_codename = 'excel_to_s3.ipynb'

In [3]:
_jiralink = 'https://hintmd.atlassian.net/browse/OTECH-2685'

In [4]:
import sys
import json
import os 
import logging
import xlrd
import pandas as pd
import numpy as np
import platform
import pytz

In [5]:
from typing import cast, Sequence, Optional, Any, Dict, List, Union
from datetime import datetime, timezone
from pprint import pprint

In [6]:
logging.basicConfig(stream=sys.stdout,level=logging.INFO)
logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.propagate = False

In [7]:
_today = datetime.today()
_today

datetime.datetime(2021, 12, 5, 12, 30, 56, 668226)

In [8]:
_todaywtz = datetime.utcnow().replace(tzinfo=pytz.utc)

In [9]:
_todaywtz

datetime.datetime(2021, 12, 5, 17, 30, 56, 679137, tzinfo=<UTC>)

In [10]:
_nowwtz = datetime.now().astimezone()

In [11]:
_nowwtz

datetime.datetime(2021, 12, 5, 12, 30, 56, 691902, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'EST'))

In [12]:
logger.info(f'test logger.info,code name:{_codename}, poc#:{_pocnumber}')

2021-12-05 12:30:56,700 __main__ INFO test logger.info,code name:excel_to_s3.ipynb, poc#:0.0.4


In [13]:
_dir = os.getcwd()

In [14]:
logger.info(f'current working directory {_dir}')

2021-12-05 12:30:56,712 __main__ INFO current working directory /Users/joe.shea/excel_to_s3


In [15]:
def get_script_path():
    return os.path.dirname(os.path.realpath(sys.argv[0]))

In [16]:
_scriptpath = get_script_path()

In [17]:
logger.info(f'script path {_scriptpath}')

2021-12-05 12:30:56,726 __main__ INFO script path /Users/joe.shea/Library/Python/3.8/lib/python/site-packages


In [18]:
logger.info(f'Python version: {platform.python_version()}')

2021-12-05 12:30:56,731 __main__ INFO Python version: 3.8.9


In [19]:
logger.info(f'pandas version {pd.__version__}')

2021-12-05 12:30:56,736 __main__ INFO pandas version 1.3.4


## ***inputs***

In [20]:
_actor = 'joe.shea@revance.com'

In [21]:
inserted_by = _actor

In [22]:
updated_by = _actor

In [23]:
inserted_comment = _jiralink

In [24]:
_file = 'OLTP_20211124.xlsx'

In [25]:
_env_type = 'OLTP'

In [26]:
_env_dbtype = 'postgres'

In [27]:
logger.info(f'data governance file:{_file}')

2021-12-05 12:30:56,771 __main__ INFO data governance file:OLTP_20211124.xlsx


In [28]:
_filesplit  =_file.split('.')

In [29]:
_filesplit

['OLTP_20211124', 'xlsx']

In [30]:
xls = pd.ExcelFile(f'{_dir}/{_file}')
df = {sh.strip():xls.parse(sh) for i, sh in enumerate(xls.sheet_names)}
df.keys()

dict_keys(['gaia', 'kronos', 'p2pe', 'merchant', 'odf', 'chargeback', 'card', 'catalog', 'payment'])

In [31]:
assert len(df.keys()) == 9 ,"expecting dict_keys(['gaia', 'kronos', 'p2pe', 'merchant', 'odf', 'chargeback', 'card', 'catalog', 'payment'])"

In [32]:
type(df)

dict

In [33]:
type(df['gaia'])

pandas.core.frame.DataFrame

In [34]:
_currentIndex = 0

In [35]:
list(df.keys())[_currentIndex ]

'gaia'

In [36]:
_currentService = list(df.keys())[_currentIndex ]

In [37]:
logger.info(f'currentService:{_currentService}')

2021-12-05 12:30:57,409 __main__ INFO currentService:gaia


In [38]:
df[_currentService].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   host                      860 non-null    object 
 1   env                       860 non-null    object 
 2   service                   860 non-null    object 
 3   table_catalog             860 non-null    object 
 4   schemaname                860 non-null    object 
 5   table_name                860 non-null    object 
 6   column_name               860 non-null    object 
 7   data_type                 860 non-null    object 
 8   character_maximum_length  228 non-null    float64
 9   sensitive_data_type       39 non-null     object 
 10  comment(s)                42 non-null     object 
dtypes: float64(1), object(10)
memory usage: 74.0+ KB


In [39]:
df[_currentService].head()

,host,env,service,table_catalog,schemaname,table_name,column_name,data_type,character_maximum_length,sensitive_data_type,comment(s)
0,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,name,text,NaN,NaN,NaN
1,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,business_id,character varying,255.0,NaN,NaN
2,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,bank_name,character varying,255.0,UNCATEGORIZED,did not see any data
3,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,bank_account,character varying,255.0,UNCATEGORIZED,did not see any data
4,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,bank_routing,character varying,255.0,UNCATEGORIZED,did not see any data


In [40]:
df[_currentService].tail()

,host,env,service,table_catalog,schemaname,table_name,column_name,data_type,character_maximum_length,sensitive_data_type,comment(s)
855,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,canceled_at,timestamp without time zone,NaN,NaN,NaN
856,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,deleted_at,timestamp without time zone,NaN,NaN,NaN
857,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,encrypted_ref_id,character varying,NaN,NaN,NaN
858,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,status,smallint,NaN,NaN,NaN
859,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,id,bigint,NaN,NaN,NaN


In [41]:
list(df[_currentService].columns)

['host',
 'env',
 'service',
 'table_catalog',
 'schemaname',
 'table_name',
 'column_name',
 'data_type',
 'character_maximum_length',
 'sensitive_data_type',
 'comment(s)']

In [42]:
df[_currentService].drop('comment(s)', axis=1, inplace=True)

In [43]:
df[_currentService].drop('service', axis=1, inplace=True)

In [44]:
_none = None

<i>__change column names to match redshift control table__</i>

<i>SELECT id, "_env_dw", "_env_type", "_env_dbtype", "_host", "_database", "_schema", "_table", "_column", "_columnlength", sensitive_data_type, is_ready_for_encryption, is_encrypted, encryption_start, encryption_end, encrypted_row_count, code_message, is_encryption_confirmed, encryption_audit_by, encryption_audit_on, encryption_audit_comment, is_ready_for_transfer, inserted_by, inserted_on, inserted_comment, updated_by, updated_on, updated_comment, "_columntype", "_columnfeed"
FROM public.encrypt_sensitive_data_ctl;<i/>

#### _rename to match control table_

In [45]:
df[_currentService].rename(columns={'host': '_host', 'env': '_env_dw','table_catalog': '_database','schemaname': '_schema','table_name': '_table','column_name': '_column','character_maximum_length': '_columnlength','sensitive_data_type': 'sensitive_data_type','data_type': '_columntype'}, inplace=True) 

In [46]:
inserted_by

'joe.shea@revance.com'

In [47]:
df[_currentService].insert(len(df[_currentService].columns), 'inserted_by', inserted_by )

In [48]:
updated_by

'joe.shea@revance.com'

In [49]:
df[_currentService].insert(len(df[_currentService].columns), 'updated_by', _none )

In [50]:
inserted_comment

'https://hintmd.atlassian.net/browse/OTECH-2685'

In [51]:
df[_currentService].insert(len(df[_currentService].columns), 'inserted_comment',  inserted_comment)

In [52]:
_env_type

'OLTP'

In [53]:
df[_currentService].insert(len(df[_currentService].columns), '_env_type',  _env_type)

In [54]:
 _env_dbtype

'postgres'

In [55]:
df[_currentService].insert(len(df[_currentService].columns), '_env_dbtype',  _env_dbtype)

In [56]:
df[_currentService].insert(len(df[_currentService].columns), 'id',  _none)

In [57]:
df[_currentService].insert(len(df[_currentService].columns), 'is_ready_for_encryption',  True)

In [58]:
df[_currentService].insert(len(df[_currentService].columns), 'is_encrypted',  False)

In [59]:
df[_currentService].insert(len(df[_currentService].columns), 'encryption_start',  _none)

In [60]:
df[_currentService].insert(len(df[_currentService].columns), 'encryption_end', _none )

In [61]:
df[_currentService].insert(len(df[_currentService].columns), 'encrypted_row_count',  _none)

In [62]:
df[_currentService].insert(len(df[_currentService].columns), 'code_message', _none )

In [63]:
df[_currentService].insert(len(df[_currentService].columns), 'is_encryption_confirmed',False  )

In [64]:
df[_currentService].insert(len(df[_currentService].columns), 'encryption_audit_by', _none )

In [65]:
df[_currentService].insert(len(df[_currentService].columns), 'encryption_audit_on', _none )

In [66]:
df[_currentService].insert(len(df[_currentService].columns), 'encryption_audit_comment', _none )

In [67]:
df[_currentService].insert(len(df[_currentService].columns), 'is_ready_for_transfer', False )

In [68]:
df[_currentService].insert(len(df[_currentService].columns), 'inserted_on', _nowwtz )

In [69]:
df[_currentService].insert(len(df[_currentService].columns), 'updated_on', _none )

In [70]:
df[_currentService].insert(len(df[_currentService].columns), 'updated_comment', _none )

In [71]:
df[_currentService].insert(len(df[_currentService].columns), '_columnfeed',  _none)

<i> __reorder columns__ </i>

In [72]:
df[_currentService] = df[_currentService][[ 'id','_env_dw','_env_type','_env_dbtype','_host', '_database','_schema','_table','_column','_columnlength','sensitive_data_type','is_ready_for_encryption','is_encrypted','encryption_start', 'encryption_end','encrypted_row_count','code_message','is_encryption_confirmed','encryption_audit_by','encryption_audit_on','encryption_audit_comment','is_ready_for_transfer','inserted_by','inserted_on','inserted_comment','updated_by','updated_on', 'updated_comment','_columntype' ,'_columnfeed']]

In [73]:
#df[_currentService] = df[_currentService][[ 'id','_env_dw','_env_type','_env_dbtype','_host', '_database','_schema','_table','_column','_columnlength','sensitive_data_type','is_ready_for_encryption','is_encrypted','encryption_start', 'encryption_end','encrypted_row_count','code_message','is_encryption_confirmed','encryption_audit_by','encryption_audit_on','encryption_audit_comment','is_ready_for_transfer','inserted_by','inserted_on','inserted_comment','_columntype']]

In [74]:
list(df[_currentService].columns)

['id',
 '_env_dw',
 '_env_type',
 '_env_dbtype',
 '_host',
 '_database',
 '_schema',
 '_table',
 '_column',
 '_columnlength',
 'sensitive_data_type',
 'is_ready_for_encryption',
 'is_encrypted',
 'encryption_start',
 'encryption_end',
 'encrypted_row_count',
 'code_message',
 'is_encryption_confirmed',
 'encryption_audit_by',
 'encryption_audit_on',
 'encryption_audit_comment',
 'is_ready_for_transfer',
 'inserted_by',
 'inserted_on',
 'inserted_comment',
 'updated_by',
 'updated_on',
 'updated_comment',
 '_columntype',
 '_columnfeed']

In [75]:
_currentService

'gaia'

In [76]:
df[_currentService] = df[_currentService][df[_currentService]['sensitive_data_type'].notnull()]

In [77]:
df[_currentService].head()

,id,_env_dw,_env_type,_env_dbtype,_host,_database,_schema,_table,_column,_columnlength,...,encryption_audit_comment,is_ready_for_transfer,inserted_by,inserted_on,inserted_comment,updated_by,updated_on,updated_comment,_columntype,_columnfeed
2,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_name,255.0,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
3,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_account,255.0,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
4,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_routing,255.0,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
5,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,identifier,255.0,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
24,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,address,name,NaN,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,text,None


In [78]:
df[_currentService].tail()

,id,_env_dw,_env_type,_env_dbtype,_host,_database,_schema,_table,_column,_columnlength,...,encryption_audit_comment,is_ready_for_transfer,inserted_by,inserted_on,inserted_comment,updated_by,updated_on,updated_comment,_columntype,_columnfeed
633,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,provider,name,NaN,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,text,None
665,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,refund,name,NaN,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,text,None
685,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,refund,reason,NaN,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,text,None
717,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,refund_transaction,last4,10.0,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
785,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,subscription,name,NaN,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,text,None


In [79]:
df[_currentService] = df[_currentService][~df[_currentService]['sensitive_data_type'].isin(['all ready encrypted'])]

In [80]:
df[_currentService].value_counts(subset=['sensitive_data_type'],dropna=False) 

sensitive_data_type
UNCATEGORIZED          23
PII                    11
PCI                     4
dtype: int64

In [81]:
df[_currentService].value_counts(subset=['_columnlength'],dropna=False) 

_columnlength
NaN              20
255.0            11
4.0               2
512.0             2
8.0               1
10.0              1
256.0             1
dtype: int64

In [82]:
df[_currentService]['_columnlength']=df[_currentService]['_columnlength'].fillna(0)

In [83]:
df[_currentService]['_columnlength']=df[_currentService]['_columnlength'].astype(np.int64)

In [84]:
df[_currentService].drop('id', axis=1, inplace=True)

In [85]:
df[_currentService].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 2 to 785
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   _env_dw                   38 non-null     object             
 1   _env_type                 38 non-null     object             
 2   _env_dbtype               38 non-null     object             
 3   _host                     38 non-null     object             
 4   _database                 38 non-null     object             
 5   _schema                   38 non-null     object             
 6   _table                    38 non-null     object             
 7   _column                   38 non-null     object             
 8   _columnlength             38 non-null     int64              
 9   sensitive_data_type       38 non-null     object             
 10  is_ready_for_encryption   38 non-null     bool               
 11  is_encrypted        

In [86]:
df[_currentService].head()

,_env_dw,_env_type,_env_dbtype,_host,_database,_schema,_table,_column,_columnlength,sensitive_data_type,...,encryption_audit_comment,is_ready_for_transfer,inserted_by,inserted_on,inserted_comment,updated_by,updated_on,updated_comment,_columntype,_columnfeed
2,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_name,255,UNCATEGORIZED,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
3,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_account,255,UNCATEGORIZED,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
4,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_routing,255,UNCATEGORIZED,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
5,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,identifier,255,UNCATEGORIZED,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,character varying,None
24,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,address,name,0,UNCATEGORIZED,...,None,False,joe.shea@revance.com,2021-12-05 12:30:56.691902-05:00,https://hintmd.atlassian.net/browse/OTECH-2685,None,None,None,text,None


In [87]:
_compressionOut ='None'

In [88]:
#_fileout_suffix = 'parquet'

In [89]:
_fileout_suffix = 'csv'

In [90]:
_fileout = _filesplit[0]+'_'+_currentService+'.'+_compressionOut+'.'+_fileout_suffix

In [91]:
logger.info(f'file out for s3:{_fileout}')

2021-12-05 12:30:57,826 __main__ INFO file out for s3:OLTP_20211124_gaia.None.csv


In [92]:
#df['gaia'].to_parquet(f'{_fileout}', compression=_compressionOut, index=False)

header = False <br>
_can keep header for debug purposes, but exclude header as part of the COPY command_

In [93]:
df[_currentService].to_csv(f'{_fileout}',index=False)

In [94]:
!ls -lrt

total 464
-rw-r--r--@ 1 joe.shea  staff     165 Nov 30 13:51 ~$OLTP_20211124.xlsx
-rw-r--r--@ 1 joe.shea  staff  188751 Dec  1 14:05 OLTP_20211124.xlsx
-rw-r--r--  1 joe.shea  staff   26216 Dec  5 12:30 excel_to_s3.ipynb
-rw-r--r--  1 joe.shea  staff   10991 Dec  5 12:30 OLTP_20211124_gaia.None.csv


In [95]:
!aws s3 cp  /Users/joe.shea/excel_to_s3 s3://com-hintmd-dev-dag/dagfile --profile mfa --recursive  --exclude "*" --include {_fileout}

upload: ./OLTP_20211124_gaia.None.csv to s3://com-hintmd-dev-dag/dagfile/OLTP_20211124_gaia.None.csv
